In [119]:
import pandas as pd
import numpy as np
import random
from collections import defaultdict
from datetime import timedelta
import math 

In [120]:

# Nombre del archivo Excel
excel_file = "generic_input_case.xlsx"

# Cargar cada hoja del Excel en un DataFrame
horizonte_df = pd.read_excel(excel_file, sheet_name="HORIZONTE")
bd_up_df = pd.read_excel(excel_file, sheet_name="BD_UP")
frota_df = pd.read_excel(excel_file, sheet_name="FROTA")
grua_df = pd.read_excel(excel_file, sheet_name="GRUA")
fabrica_df = pd.read_excel(excel_file, sheet_name="FABRICA")
rota_df = pd.read_excel(excel_file, sheet_name="ROTA")

horizonte_df=horizonte_df.iloc[0:5]
fabrica_df=fabrica_df.iloc[0:5]

# Mostrar las primeras filas de cada DataFrame para verificar la carga
print("HORIZONTE:")
print(horizonte_df.head())
print("\nBD_UP:")
print(bd_up_df.head())
print("\nFROTA:")
print(frota_df.head())
print("\nGRUA:")
print(grua_df.head())
print("\nFABRICA:")
print(fabrica_df.head())
print("\nROTA:")
print(rota_df.head())

HORIZONTE:
   DIA  MES   ANO CICLO_LENTO
0    1    5  2020           X
1    2    5  2020         NaN
2    3    5  2020         NaN
3    4    5  2020         NaN
4    5    5  2020         NaN

BD_UP:
       UP                      FAZENDA          DB    VOLUME       RSP  \
0  S6C421                      SIRIEMA  406.666302   4500.00  1.289003   
1  S3AX03                   INDIANA II  410.081889   2124.07  1.302226   
2  S6C297                    FORTALEZA  408.376272   4791.00  1.367399   
3  S5AW14  PIRACEMA_GLEBA PULADOR - DX  467.553351  18533.08  1.378530   
4  S5AW09  PIRACEMA_GLEBA PULADOR - DX  485.300233   3320.00  1.403137   

  DATA_COLHEITA  IDADE_FLORESTA        IMA  RD RESERVADO  CLONE  ESPECIE  \
0    2020-01-07        5.889117  43.769052 NaN  RSP_ALTO    NaN      NaN   
1    2020-01-15        6.036961  40.769956 NaN  RSP_ALTO    NaN      NaN   
2    2020-03-10        5.415469  33.664280 NaN  RSP_ALTO    NaN      NaN   
3    2020-02-01        9.598905  38.702981 NaN  RSP_

c:\Users\wilma\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:81: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


In [121]:
def generar_solucion_inicial(horizonte_df, bd_up_df, frota_df, rota_df, fabrica_df):
    """
    Genera una solución inicial parcialmente factible priorizando la demanda y capacidad.

    Args:
        horizonte_df (pd.DataFrame): Datos del horizonte de planificación.
        bd_up_df (pd.DataFrame): Datos de las UPs.
        frota_df (pd.DataFrame): Datos de la flota de transportadores.
        rota_df (pd.DataFrame): Datos de las rutas.
        fabrica_df (pd.DataFrame): Datos de la fábrica y su demanda.

    Returns:
        dict: Una solución inicial en forma de diccionario.
              La clave es (DIA, FABRICA, UP, TRANSPORTADOR) y el valor es la cantidad de camiones.
    """

    solucion = {}
    for _, dia_row in horizonte_df.iterrows():
        dia = dia_row['DIA']
        fabrica = fabrica_df.iloc[dia-1]['FABRICA']
        demanda_min = fabrica_df.iloc[dia-1]['DEMANDA_MIN']
        demanda_max = fabrica_df.iloc[dia-1]['DEMANDA_MAX']
        demanda_actual = 0

        # Filtrar UPs y rutas que pueden servir a esta fábrica
        ups_disponibles = bd_up_df.index.tolist()  # Usar el índice para identificar UPs
        rutas_posibles = rota_df[(rota_df['DESTINO'] == fabrica) & (rota_df['ORIGEM'].isin(bd_up_df['UP']))].sample(frac=1).sort_values(by=["Fazenda"])

        transportadores_disponibles = rutas_posibles['TRANSPORTADOR'].unique().tolist()
        pending={}
        gruas_d={}
        for _, t in frota_df.iterrows():
            # print(t)
            key=t["TRANSPORTADOR"]
            pending[key]= random.randint(t['FROTA_MIN'],t['FROTA_MAX'])
        for _, t in grua_df.iterrows():
            # print(t)
            key=t["TRANSPORTADOR"]        
            gruas_d[key]= t["QTD_GRUAS"]
        maxx_iter=0
        while (demanda_actual < demanda_min):  # Intentar satisfacer la demanda mínima
            maxx_iter+=1
            if maxx_iter>=len(ups_disponibles)*len(rutas_posibles)*len(rutas_posibles['TRANSPORTADOR'].unique().tolist()):
                break
            # Seleccionar UP y transportador aleatoriamente
            up_idx = random.choice(ups_disponibles)
            up = bd_up_df.loc[up_idx]['UP']
            transportador = random.choice(transportadores_disponibles)

            perc=grua_df[grua_df["TRANSPORTADOR"]==transportador].iloc[0]["PORCENTAGEM_VEICULOS_MIN"]

            if not ups_disponibles or not transportadores_disponibles:
                break  # No hay más UPs o transportadores disponibles
            if gruas_d[transportador]==0:
                continue
            # Verificar si existe una ruta válida
            if not rutas_posibles[(rutas_posibles['ORIGEM'] == up) & (rutas_posibles['TRANSPORTADOR'] == transportador)].empty:
                ruta = rutas_posibles[(rutas_posibles['ORIGEM'] == up) & (rutas_posibles['TRANSPORTADOR'] == transportador)].iloc[0]

                # Calcular cuántos camiones se pueden enviar
                carga_camion = ruta['CAIXA_CARGA']
                ciclos = ruta['TEMPO_CICLO']
                volumen_up = bd_up_df.loc[up_idx]['VOLUME']
                max_camiones = min(int(volumen_up / (carga_camion*ciclos)), 
                                   random.randint(perc*100,100)/100*frota_df[frota_df['TRANSPORTADOR'] == transportador]['FROTA_MAX'].max())

                n_camiones=math.ceil(max(max_camiones,perc*frota_df[frota_df['TRANSPORTADOR'] == transportador]['FROTA_MAX'].max()))
                
                if n_camiones > (1-perc)*frota_df[frota_df['TRANSPORTADOR'] == transportador]['FROTA_MAX'].max():
                    n_camiones=frota_df[frota_df['TRANSPORTADOR'] == transportador]['FROTA_MAX'].max()
                if gruas_d[transportador]==1:
                    n_camiones=math.ceil(pending[transportador])

                if n_camiones > 0:
                    solucion[(dia, fabrica, up, transportador)] = solucion.get((dia, fabrica, up, transportador), 0) + n_camiones
                    demanda_actual += n_camiones * carga_camion * ciclos
                    pending[transportador]-=n_camiones
                    gruas_d[transportador]-=1
                    rutas_posibles=rutas_posibles.drop(rutas_posibles[(rutas_posibles['ORIGEM'] == ruta["ORIGEM"])  & (rutas_posibles['TRANSPORTADOR'] == ruta["TRANSPORTADOR"])].index.to_list())

        
        print ("día",dia)
        print("demanda atentida",demanda_actual)
        if not (demanda_actual >= demanda_min) and (demanda_actual <=demanda_max):
            print(f"Satisfacción de demanda de UP: {up} No cumplida, demanda actual: {demanda_actual}")

    return solucion



In [122]:
solucion_inicial = generar_solucion_inicial(horizonte_df, bd_up_df, frota_df, rota_df, fabrica_df)
print("\nSolución Inicial:")
print(solucion_inicial)

día 1
demanda atentida 6375.6
Satisfacción de demanda de UP: S5AW05 No cumplida, demanda actual: 6375.6
día 2
demanda atentida 7002.6
día 3
demanda atentida 6573.599999999999
día 4
demanda atentida 6850.800000000001
día 5
demanda atentida 7108.200000000001

Solución Inicial:
{(1, 'LIM', 'S5AK08', 'Rampazo'): 21, (1, 'LIM', 'S5AK09', 'Pastori'): 27, (1, 'LIM', 'S6C334', 'Tover'): 14, (2, 'LIM', 'S5AW14', 'Pastori'): 27, (2, 'LIM', 'S5AW05', 'Rampazo'): 12, (2, 'LIM', 'S6BG08', 'Tover'): 14, (2, 'LIM', 'S3AX06', 'Rampazo'): 7, (3, 'LIM', 'S5AW13', 'Pastori'): 27, (3, 'LIM', 'S6BG14', 'Tover'): 6, (3, 'LIM', 'S6C335', 'Tover'): 7, (3, 'LIM', 'S5AK08', 'Rampazo'): 21, (4, 'LIM', 'S3AX03', 'Rampazo'): 21, (4, 'LIM', 'S5AW13', 'Pastori'): 16, (4, 'LIM', 'S6C334', 'Tover'): 14, (4, 'LIM', 'S5AK05', 'Pastori'): 9, (5, 'LIM', 'S5AW10', 'Pastori'): 27, (5, 'LIM', 'S3AX02', 'Rampazo'): 21, (5, 'LIM', 'S6C298', 'Tover'): 14}


In [123]:
def verificar_demanda(solucion, fabrica_df, rota_df):
    """Verifica si se cumple la demanda diaria de la fábrica."""
    for _, fabrica_row in fabrica_df.iterrows():
        dia = fabrica_row['DIA']
        fabrica = fabrica_row['FABRICA']
        demanda_min = fabrica_row['DEMANDA_MIN']
        demanda_max = fabrica_row['DEMANDA_MAX']
        demanda_actual = 0
        for (d, f, up, _), camiones in solucion.items():
            if d == dia and f == fabrica:
                # Buscar la caja de carga correspondiente
                carga_camion = rota_df[(rota_df['ORIGEM'] == up) & (rota_df['DESTINO'] == f) ]['CAIXA_CARGA'].iloc[0]
                ciclos = rota_df[(rota_df['ORIGEM'] == up) & (rota_df['DESTINO'] == f) ]["TEMPO_CICLO"].iloc[0]
                demanda_actual += camiones * carga_camion * ciclos
        if not (demanda_min <= demanda_actual <= demanda_max):
            print(f"Demanda no cumplida para Fábrica {fabrica} en el Día {dia}: {demanda_actual}")
            return False
    return True

def verificar_calidad_rsp(solucion, bd_up_df, fabrica_df, rota_df):
    """Verifica si la media ponderada de RSP está dentro de los límites."""

    for _, fabrica_row in fabrica_df.iterrows():
        dia = fabrica_row['DIA']
        fabrica = fabrica_row['FABRICA']
        rsp_min = fabrica_row['RSP_MIN']
        rsp_max = fabrica_row['RSP_MAX']

        volumen_total = 0
        rsp_ponderado_total = 0

        for (d, f, up, _), camiones in solucion.items():
            if d == dia and f == fabrica:
                # Obtener el RSP y la caja de carga para esta UP
                rsp_up = bd_up_df[bd_up_df['UP'] == up]['RSP'].iloc[0]
                carga_camion = rota_df[(rota_df['ORIGEM'] == up) & (rota_df['DESTINO'] == fabrica)]['CAIXA_CARGA'].iloc[0]
                ciclos = rota_df[(rota_df['ORIGEM'] == up) & (rota_df['DESTINO'] == fabrica) ]["TEMPO_CICLO"].iloc[0]
                volumen_envio = camiones * carga_camion * ciclos
                volumen_total += volumen_envio
                rsp_ponderado_total += rsp_up * volumen_envio

        if volumen_total > 0:  # Evitar división por cero
            rsp_promedio = rsp_ponderado_total / volumen_total
            if not (rsp_min <= round(rsp_promedio,2) <= rsp_max):
                print(f"RSP fuera de rango para Fábrica {fabrica} en el Día {dia}: {rsp_promedio}")
                return False
        else:
            print(f"No hay envíos para Fábrica {fabrica} en el Día {dia} para verificar RSP")
    return True

def verificar_capacidad_vehiculos(solucion, rota_df, frota_df):
    """Verifica si se respeta la cantidad de los vehículos."""
    for (dia, _, _, transportador), camiones in solucion.items():
        # Filtrar la flota para el transportador y el día específico (si está presente) o tomar el valor general
        frota_dia = frota_df[(frota_df['TRANSPORTADOR'] == transportador) & (frota_df['DIA'] == dia)]
        if not frota_dia.empty:
            max_camiones = frota_dia['FROTA_MAX'].iloc[0]
        else:
            max_camiones = frota_df[(frota_df['TRANSPORTADOR'] == transportador) & (frota_df['DIA'].isna())]['FROTA_MAX'].max()

        if camiones > max_camiones:
            print(f"Capacidad de vehículos excedida para Transportador {transportador} en el Día {dia}: {camiones} > {max_camiones}")
            return False
    return True

def verificar_capacidad_gruas(solucion, grua_df, solucion_dict, bd_up_df):
    """Verifica si se respeta la cantidad de las grúas."""

    for transportador in grua_df['TRANSPORTADOR'].unique():
        qtd_gruas = grua_df[grua_df['TRANSPORTADOR'] == transportador]['QTD_GRUAS'].iloc[0]

        for dia in horizonte_df['DIA']:
            ups_atendidas = set()
            for (d, _, up, t), camiones in solucion_dict.items():
                if d == dia and t == transportador:
                    ups_atendidas.add(up)

            if len(ups_atendidas) > qtd_gruas:
                print(f"Cantidad de grúas excedida para Transportador {transportador} en el Día {dia}: {len(ups_atendidas)} > {qtd_gruas}")
                return False
    return True

def verificar_asignacion_transportador_fazenda(solucion, rota_df):
    """Verifica si un transportador no opera en dos fazendas distintas el mismo día."""

    asignaciones_diarias = {}
    for (dia, _, up, transportador), _ in solucion.items():
        fazenda = rota_df[rota_df['ORIGEM'] == up]['Fazenda'].iloc[0]  # Obtener la fazenda de la UP
        if (dia, transportador) not in asignaciones_diarias:
            asignaciones_diarias[(dia, transportador)] = set()
        asignaciones_diarias[(dia, transportador)].add(fazenda)

    for _, fazendas in asignaciones_diarias.items():
        if len(fazendas) > 1:
            print(f"Más de 1 Fazenda visita por el Transportador {transportador} en el Día {dia}")
            return False
    return True

def verificar_consumo_recursos(solucion, frota_df, horizonte_df):
    """Verifica si se respetan los límites mínimo y máximo de equipos asignables a cada transportador."""

    for transportador in frota_df['TRANSPORTADOR'].unique():
        for dia in horizonte_df['DIA']:
            frota_min = frota_df[(frota_df["TRANSPORTADOR"] == transportador)]['FROTA_MIN'].max()
            frota_max = frota_df[(frota_df["TRANSPORTADOR"] == transportador)]['FROTA_MAX'].max()


            camiones_usados = 0
            for (d, _, _, t), camiones in solucion.items():
                if d == dia and t == transportador:
                    camiones_usados += camiones

            if not (frota_min <= camiones_usados <= frota_max):
                print(f"Consumo de recursos fuera de rango para Transportador {transportador} en el Día {dia}: {camiones_usados}, cuando el mínimo es {frota_min} y el máximo es {frota_max}")
                return False
    return True

def verificar_porcentaje_vehiculos_grua(solucion, grua_df, bd_up_df, horizonte_df):
    """Verifica si el número de vehículos asignados a cada UP cumple con el porcentaje mínimo respecto al total de vehículos en actividad cada día."""

    for dia in horizonte_df['DIA']:
        total_camiones_dia = 0
        camiones_por_transportador = {}

        # Contar el total de camiones por día y los camiones por transportador
        for (d, _, _, t), camiones in solucion.items():
            if d == dia:
                total_camiones_dia += camiones
                camiones_por_transportador[t] = camiones_por_transportador.get(t, 0) + camiones

        for up in bd_up_df['UP']:
            camiones_up_dia = 0
            transportadores_up = set()  # Para almacenar los transportadores que sirven a esta UP en este día

            for (d, _, u, t), camiones in solucion.items():
                if d == dia and u == up:
                    camiones_up_dia += camiones
                    transportadores_up.add(t)

            if total_camiones_dia > 0:
                for transportador in transportadores_up:
                    porcentaje_minimo = grua_df[grua_df['TRANSPORTADOR'] == transportador]['PORCENTAGEM_VEICULOS_MIN'].iloc[0]
                    if camiones_up_dia / camiones_por_transportador.get(transportador, 1) < porcentaje_minimo:  # Usar 1 como valor por defecto para evitar división por cero
                        print(f"Porcentaje mínimo de vehículos no cumplido para UP {up} por Transportador {transportador} en el Día {dia}: {camiones_up_dia / camiones_por_transportador.get(transportador, 1)}")
                        return False
            elif camiones_up_dia > 0:
                print(f"Total camiones dia 0, camiones up dia {camiones_up_dia}")
                return False
    return True


def verificar_transporte_up(solucion, bd_up_df):
    """
    Verifica las restricciones de transporte completo (UPs < 7000 m³) y fraccionado (UPs >= 7000 m³).

    Args:
        solucion (dict): La solución actual.
        bd_up_df (pd.DataFrame): Datos de las UPs.

    Returns:
        bool: True si se cumplen las restricciones, False en caso contrario.
    """

    estado_ups = {}  # Para rastrear el estado de transporte de cada UP

    for (dia, _, up, _), _ in solucion.items():
        volumen_up = bd_up_df[bd_up_df['UP'] == up]['VOLUME'].iloc[0]

        if up not in estado_ups:
            estado_ups[up] = {
                'volumen': volumen_up,
                'entradas': 0,
                'transporte_iniciado': False,
                'ultimo_dia_transporte': None,
                'intervalo_permitido': True  # Inicialmente, se permite el intervalo
            }

        if not estado_ups[up]['transporte_iniciado']:
            estado_ups[up]['transporte_iniciado'] = True
            estado_ups[up]['entradas'] += 1
        else:
            if dia != estado_ups[up]['ultimo_dia_transporte'] and estado_ups[up]['ultimo_dia_transporte'] is not None:
              if dia - estado_ups[up]['ultimo_dia_transporte'] > 1: #Hubo un intervalo
                if estado_ups[up]['intervalo_permitido']:
                  estado_ups[up]['entradas'] += 1
                  estado_ups[up]['intervalo_permitido'] = False
                else:
                  print(f"Más de dos entradas para UP {up}")
                  return False

        if estado_ups[up]['entradas'] > 2:
            print(f"Más de dos entradas para UP {up}")
            return False

        estado_ups[up]['ultimo_dia_transporte'] = dia

    # Verificar transporte completo para UPs pequeñas
    for up, estado in estado_ups.items():
        if estado['volumen'] < 7000 and estado['entradas'] > 1:
            print(f"Transporte fraccionado para UP pequeña {up}")
            return False

    return True


In [124]:
def es_solucion_factible(solucion, horizonte_df, bd_up_df, frota_df, grua_df, fabrica_df, rota_df):
    """
    Verifica si una solución cumple con todas las restricciones del problema.

    Args:
        solucion (dict): La solución a verificar.
        horizonte_df (pd.DataFrame): Datos del horizonte de planificación.
        bd_up_df (pd.DataFrame): Datos de las UPs.
        frota_df (pd.DataFrame): Datos de la flota.
        grua_df (pd.DataFrame): Datos de las grúas.
        fabrica_df (pd.DataFrame): Datos de la fábrica.
        rota_df (pd.DataFrame): Datos de las rutas.

    Returns:
        bool: True si la solución es factible, False en caso contrario.
    """

    factible = True

    # 1. Demanda diaria de la fábrica
    if not verificar_demanda(solucion, fabrica_df, rota_df):
        factible = False
        print("Falla demanda")

    # 2. Calidad de la madera (RSP)
    if not verificar_calidad_rsp(solucion, bd_up_df, fabrica_df, rota_df):
        factible = False
        print("Falla RSP")

    # 3. Capacidad de los vehículos
    if not verificar_capacidad_vehiculos(solucion, rota_df, frota_df):
        factible = False
        print("Falla capacidad vehiculos")

    # 4. Capacidad de las grúas
    if not verificar_capacidad_gruas(solucion, grua_df, solucion, bd_up_df):
        factible = False
        print("Falla capacidad gruas")

    # 5. Asignación Transportador x Fazenda
    if not verificar_asignacion_transportador_fazenda(solucion, rota_df):
        factible = False
        print("Falla asignacion fazenda")

    # 6. Consumo de recursos (flota)
    if not verificar_consumo_recursos(solucion, frota_df, horizonte_df):
        factible = False
        print("Falla consumo recursos")

    # 7. Grúas (porcentaje mínimo de vehículos por UP)
    if not verificar_porcentaje_vehiculos_grua(solucion, grua_df, bd_up_df, horizonte_df):
       factible = False
       print("Falla porcentaje vehiculos grua")

    # 9. Transporte completo (UPs < 7000 m³) y Fraccionado (UPs > 7000 m³)
    if not verificar_transporte_up(solucion, bd_up_df):
        factible = False
        print("Falla transporte up")

    return factible

In [125]:
# Verificar la factibilidad de la solución inicial
es_factible = es_solucion_factible(solucion_inicial, horizonte_df, bd_up_df, frota_df, grua_df, fabrica_df, rota_df)
print("\n¿Es la solución inicial factible?:", es_factible)

Demanda no cumplida para Fábrica LIM en el Día 1: 6375.6
Falla demanda
RSP fuera de rango para Fábrica LIM en el Día 1: 1.5115545027099404
Falla RSP
Más de 1 Fazenda visita por el Transportador Tover en el Día 5
Falla asignacion fazenda
Porcentaje mínimo de vehículos no cumplido para UP S3AX06 por Transportador Rampazo en el Día 2: 0.3684210526315789
Falla porcentaje vehiculos grua
Transporte fraccionado para UP pequeña S6C334
Falla transporte up

¿Es la solución inicial factible?: False


In [126]:
def calcular_variacion_db(solucion, bd_up_df, horizonte_df):
    """
    Calcula la variación diaria de la densidad básica (DB).

    Args:
        solucion (dict): La solución actual.
        bd_up_df (pd.DataFrame): Datos de las UPs.
        horizonte_df (pd.DataFrame): Datos del horizonte de planificación.

    Returns:
        float: La máxima variación de DB encontrada en el horizonte de planificación.
    """
    max_variacion_db = 0
    for _, dia_row in horizonte_df.iterrows():
        dia = dia_row['DIA']
        dbs_dia = []
        volumenes_dia = []
        for (d, _, up, _), camiones in solucion.items():
            if d == dia:
                db_up = bd_up_df[bd_up_df['UP'] == up]['DB'].iloc[0]
                carga_camion = rota_df[(rota_df['ORIGEM'] == up) & (rota_df['DESTINO'] == 'LIM')]['CAIXA_CARGA'].iloc[0]
                ciclo = rota_df[(rota_df['ORIGEM'] == up) & (rota_df['DESTINO'] == 'LIM')]['TEMPO_CICLO'].iloc[0]
                volumen_transportado = camiones * carga_camion * ciclo
                dbs_dia.extend([db_up] * int(volumen_transportado))  # Añadir DB repetida según el volumen

        if dbs_dia:
            max_db_dia = max(dbs_dia)
            min_db_dia = min(dbs_dia)
            variacion_db_dia = max_db_dia - min_db_dia
            max_variacion_db = max(max_variacion_db, variacion_db_dia)
        else:
            max_variacion_db = 0 #Si no hay envios en el día la variacion es 0
    return max_variacion_db


In [127]:
# Calcular la variación de DB de la solución inicial
variacion_db_inicial = calcular_variacion_db(solucion_inicial, bd_up_df, horizonte_df)
print("\nVariación DB de la solución inicial:", variacion_db_inicial)



Variación DB de la solución inicial: 83.12135851554524


In [128]:

def generar_vecindad(solucion, bd_up_df, rota_df):
    """
    Genera una solución vecina a partir de la solución actual.  La vecindad se define
    como un pequeño cambio en la cantidad de camiones asignados a una UP y transportador en un día.

    Args:
        solucion (dict): La solución actual.
        bd_up_df (pd.DataFrame): Datos de las UPs.
        rota_df (pd.DataFrame): Datos de las rutas.

    Returns:
        dict: Una nueva solución vecina.
    """
    vecino = solucion.copy()
    if not vecino:
        return vecino

    # Seleccionar aleatoriamente un día, fábrica, UP y transportador
    dia, fabrica, up, transportador = random.choice(list(vecino.keys()))

    # Obtener información de la UP y la ruta
    volumen_up = bd_up_df[bd_up_df['UP'] == up]['VOLUME'].iloc[0]
    carga_camion = rota_df[(rota_df['ORIGEM'] == up) & (rota_df['DESTINO'] == fabrica)]['CAIXA_CARGA'].iloc[0]
    ciclo= rota_df[(rota_df['ORIGEM'] == up) & (rota_df['DESTINO'] == fabrica)]['TEMPO_CICLO'].iloc[0]
    max_camiones_up = int(volumen_up / (carga_camion*ciclo))
    
    # Calcular la cantidad actual de camiones para esta asignación
    cantidad_actual = vecino.get((dia, fabrica, up, transportador), 0)

    # Definir un rango de cambio para la cantidad de camiones
    cambio_maximo = 2  # Puedes ajustar este valor
    cambio = random.randint(-cambio_maximo, cambio_maximo)

    nueva_cantidad = cantidad_actual + cambio
    
    # Asegurar que la nueva cantidad no sea negativa y no exceda el máximo posible
    nueva_cantidad = max(0, min(nueva_cantidad, max_camiones_up))
    
    if nueva_cantidad > 0:
        vecino[(dia, fabrica, up, transportador)] = nueva_cantidad
    elif (dia, fabrica, up, transportador) in vecino:
        del vecino[(dia, fabrica, up, transportador)] #Eliminar la entrada si queda en 0

    return vecino



In [129]:
def busqueda_tabu(solucion_inicial, horizonte_df, bd_up_df, frota_df, grua_df, fabrica_df, rota_df, max_iteraciones=10, tamano_lista_tabu=10):
    """
    Implementa el algoritmo de Búsqueda Tabú para encontrar una solución óptima o近似ada
    al problema de secuenciación de transporte de madera.

    Args:
        solucion_inicial (dict): Una solución inicial factible.
        horizonte_df (pd.DataFrame): Datos del horizonte de planificación.
        bd_up_df (pd.DataFrame): Datos de las UPs.
        frota_df (pd.DataFrame): Datos de la flota de transportadores.
        grua_df (pd.DataFrame): Datos de las grúas.
        fabrica_df (pd.DataFrame): Datos de la fábrica.
        rota_df (pd.DataFrame): Datos de las rutas.
        max_iteraciones (int): Número máximo de iteraciones a realizar.
        tamano_lista_tabu (int): Tamaño de la lista tabú.

    Returns:
        dict: La mejor solución encontrada.
        float: El valor de la función objetivo (variación de DB) de la mejor solución.
    """
    solucion_actual = solucion_inicial
    mejor_solucion = solucion_inicial
    mejor_variacion_db = calcular_variacion_db(solucion_inicial, bd_up_df, horizonte_df)
    lista_tabu = []
    
    iteracion = 0
    while iteracion < max_iteraciones:
        vecino_mejor = None
        vecino_mejor_variacion = float('inf')
        
        # Generar vecindad
        for _ in range(10):  # Generar varios vecinos y elegir el mejor
            vecino = generar_vecindad(solucion_actual, bd_up_df, rota_df)
            if not es_solucion_factible(vecino, horizonte_df, bd_up_df, frota_df, grua_df, fabrica_df, rota_df):
                continue  # Saltar vecinos no factibles
            
            variacion_db_vecino = calcular_variacion_db(vecino, bd_up_df, horizonte_df)
            
            if vecino not in lista_tabu and variacion_db_vecino < vecino_mejor_variacion:
                vecino_mejor = vecino
                vecino_mejor_variacion = variacion_db_vecino
                
        if vecino_mejor is None:
            print(f"Iteración {iteracion}: No se encontraron vecinos factibles")
            iteracion+=1
            continue  # No hay vecinos factibles

        # Actualizar la lista tabú
        lista_tabu.append(vecino_mejor)
        if len(lista_tabu) > tamano_lista_tabu:
            lista_tabu.pop(0)  # Eliminar el elemento más antiguo

        # Actualizar la solución actual
        solucion_actual = vecino_mejor

        # Actualizar la mejor solución si se encuentra una mejor
        if vecino_mejor_variacion < mejor_variacion_db:
            mejor_solucion = vecino_mejor
            mejor_variacion_db = vecino_mejor_variacion
            print(f"Iteración {iteracion}: ¡Mejor solución encontrada! Variación DB: {mejor_variacion_db}")
        else:
            print(f"Iteración {iteracion}: Solución no mejorada. Variación DB: {vecino_mejor_variacion}")
        iteracion += 1
    
    return mejor_solucion, mejor_variacion_db

In [130]:
# Generar una solución inicial
solucion_inicial = generar_solucion_inicial(horizonte_df, bd_up_df, frota_df, rota_df, fabrica_df)
print("\nSolución Inicial:")
print(solucion_inicial)

# Verificar la factibilidad de la solución inicial

es_factible = es_solucion_factible(solucion_inicial, horizonte_df, bd_up_df, frota_df, grua_df, fabrica_df, rota_df)
print("\n¿Es la solución inicial factible?:", es_factible)



día 1
demanda atentida 7180.8
día 2
demanda atentida 7576.8
día 3
demanda atentida 6692.4
día 4
demanda atentida 7062.0
día 5
demanda atentida 6494.4

Solución Inicial:
{(1, 'LIM', 'S3AX02', 'Rampazo'): 12, (1, 'LIM', 'S6BG12', 'Tover'): 8, (1, 'LIM', 'S3AX04', 'Pastori'): 11, (1, 'LIM', 'S3AX02', 'Pastori'): 15, (1, 'LIM', 'S6C334', 'Tover'): 4, (1, 'LIM', 'S5AW14', 'Rampazo'): 8, (2, 'LIM', 'S3AX06', 'Rampazo'): 21, (2, 'LIM', 'S6BG13', 'Tover'): 14, (2, 'LIM', 'S5AW10', 'Pastori'): 16, (2, 'LIM', 'S3AX04', 'Pastori'): 10, (3, 'LIM', 'S5AW09', 'Rampazo'): 21, (3, 'LIM', 'S6C335', 'Tover'): 14, (3, 'LIM', 'S5AW05', 'Pastori'): 27, (4, 'LIM', 'S5AW13', 'Pastori'): 27, (4, 'LIM', 'S6BG12', 'Tover'): 14, (4, 'LIM', 'S3AX01', 'Rampazo'): 10, (4, 'LIM', 'S5AW05', 'Rampazo'): 9, (5, 'LIM', 'S5AW05', 'Rampazo'): 9, (5, 'LIM', 'S5AW10', 'Pastori'): 27, (5, 'LIM', 'S5AW10', 'Rampazo'): 10, (5, 'LIM', 'S6C298', 'Tover'): 14}
RSP fuera de rango para Fábrica LIM en el Día 1: 1.5129891870203038
Fa

In [131]:
# Calcular la variación de DB de la solución inicial
variacion_db_inicial = calcular_variacion_db(solucion_inicial, bd_up_df, horizonte_df)
print("\nVariación DB de la solución inicial:", variacion_db_inicial)

# # Ejecutar la Búsqueda Tabú
# mejor_solucion, mejor_variacion_db = busqueda_tabu(solucion_inicial, horizonte_df, bd_up_df, frota_df, grua_df, fabrica_df, rota_df, max_iteraciones=10, tamano_lista_tabu=15)

# print("\nMejor Solución Encontrada:")
# print(mejor_solucion)
# print("\nVariación DB de la Mejor Solución:", mejor_variacion_db)


Variación DB de la solución inicial: 39.657814115676274
